In [ ]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import glob, os
from scipy.signal import savgol_filter
import numpy as np
import seaborn as sn
%load_ext tensorboard
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.models import load_model
import tf2onnx
import onnx
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from joblib import dump, load
from keras.models import Sequential
from keras.layers import Input, LSTM,Dense, GaussianNoise, BatchNormalization, Activation, PReLU, Dropout
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
from keras.optimizers import nadam_v2, rmsprop_v2, adamax_v2
from keras.activations import silu,swish, linear, tanh,leaky_relu, sigmoid, relu,elu, selu, gelu, linear
from keras.regularizers import l1, l2, L1L2
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
source_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\New_Experiments\\ki_modell\\t_welle\\data\\'
model_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\New_Experiments\\ki_modell\\t_welle\\modell\\'
preprocessor_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\New_Experiments\\ki_modell\\t_welle\\preprocessor\\'

In [ ]:
from keras.losses import LossFunctionWrapper, mean_absolute_error
import tensorflow as tf
import keras.backend as k

def max_loss(y_true: tf.Tensor, y_pred: tf.Tensor):
    #error= tf.multiply(tf.add(tf.multiply(tf.reduce_max(tf.abs((y_true - y_pred))),0.2) , mean_absolute_error(y_true , y_pred)),1000)
    error= tf.add(tf.multiply(tf.reduce_max(tf.abs((y_true - y_pred))),1) , mean_absolute_error(y_true , y_pred))
    return error
special_loss= LossFunctionWrapper(max_loss)

In [ ]:
def generateDataSource(signal_input= None, input_columns: list = [], output_length: int=1, signal_output= None, window= 1 , shift= 1):
    Signal_Length = signal_input.shape[0]
    num_samples= int((Signal_Length - window +1 ) / shift)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y= np.zeros(shape=(num_samples, output_length, 1))
    for i in range (num_samples):
        x[i]= signal_input[i * shift : i * shift + window]
        y[i]= signal_output[i * shift + window - 1 : i * shift + window - 1+ output_length]
    return x, y

In [ ]:

def build_Model( X_train,y_train):
    activation_Function= tanh
    dropout= 0.0
    l1_v= 0.00001
    L2_v= 0.00001
    size= 10
    n_hidden_layers=4
    unroll= False
    optimizer= nadam_v2.Nadam()#
    model= Sequential()
    model.add(LSTM(size, input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences= True))#,unroll= unroll, dropout=dropout, kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(Activation(activation= activation_Function))
    #model.add(BatchNormalization())
    #model.add(PReLU())
    for i in range(n_hidden_layers):
      model.add(LSTM(size, return_sequences= True))#, unroll= unroll, dropout=dropout, kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
      #model.add(BatchNormalization())
      model.add(Activation(activation= activation_Function))
      #model.add(BatchNormalization())
      #model.add(PReLU())
    model.add(LSTM(size, return_sequences= False))#, unroll= unroll, dropout=dropout, kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(Activation(activation= activation_Function))
    #model.add(BatchNormalization())
    #model.add(PReLU())
    model.add(Dense(1,activation='linear'))
    model.compile(optimizer= optimizer, loss= special_loss , metrics=[mae])
    return model

In [ ]:
####MLP#####
def build_Model_MLP(X_train,y_train, batch_size= 1):
    dropout= 0.0
    l1_v= 0.00001
    L2_v= 0.00001
    activation= relu
    optimizer= nadam_v2.Nadam(learning_rate=0.001)
    model= Sequential()
    model.add(Dense(50, input_shape =  (X_train.shape[1],X_train.shape[2])))#,kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(dropout))
    model.add(Dense(50))#, kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(dropout))
    model.add(Dense(50))#, kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))#
    #model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(dropout))
    model.add(Dense(50))#,kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(dropout))
    model.add(Dense(50))#,kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(dropout))
    model.add(Dense(50))#,kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(dropout))
    model.add(Dense(50))#,kernel_regularizer=L1L2(l1= l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='linear'))
    model.compile(optimizer= optimizer, loss= max_loss , metrics=[mae])
    return model

In [ ]:
### Implement special preprocessing steps 
### 1 calculate new Feature
### 2 make rounding


from sklearn.preprocessing import FunctionTransformer
def special_preprocess(data: np.ndarray):
    delta_t_spindel_bett= data[:,1] - data[:,0]
    delta_t_spindel_bett= delta_t_spindel_bett.reshape((-1,1))
    result= np.concatenate(( data[:,2:5], delta_t_spindel_bett), axis=1)
    return result

def rounding(signals: np.ndarray):
    return np.round(signals,3)


In [ ]:
#selected_Columns= ['diff', 'drhz','t_bett',]#,'t_kluelung']# , 'diff'
# [ 't_bett','t_spindel', 't_y_achse', 't_z_achse', 'drhz', 't_motor','t_kss_tank', 't_raum', 't_spindel_vor', 't_spindel_rueck','t_schwenkantrieb']
scaler= StandardScaler(with_mean= True, with_std= True)
minmax= MinMaxScaler(feature_range=(-1,1))
pca= PCA(n_components=2)
##Data Parameters
selected_Columns= ['t_motor']
window=15
shift= 1
proposed_pipline= Pipeline(steps=[('special', FunctionTransformer(rounding))])# #Pipeline(steps=[('stdscaler', scaler), ('pca', pca)])# ('stdscaler', scaler),
preprocessor_name= "preprocessor.p"

In [ ]:
def visualize_data(df:pd.DataFrame, name: str, features: list):
    fig, ax= plt.subplots(figsize=(15,15))
    print(name)
    plt.title(name)
    plt.grid()
    i= 0
    for feature in features:
        i+=1
        plt.subplot(7,1,i)
        plt.plot(df[feature], label= feature)
        plt.legend() 
    plt.show()
    plt.close()

In [ ]:
def create_histogram(df:pd.DataFrame, name: str, features: list):
    fig, ax= plt.subplots(figsize=(15,15))
    plt.title(name)
    plt.grid()
    i= 0
    for feature in features:
        i+=1
        plt.subplot(7,1,i)
        plt.hist(df[feature], label= feature)
        plt.legend() 
    plt.show()
    plt.close()

In [ ]:
# from pymongo import MongoClient
# #### Read LF data ##New data structure
# host= "localhost"
# port=27017
# client = MongoClient(host= host, port=port)
# db = client.h4s
# event_list = db.lf_data.find().sort('date', 1)
# lf_signals= None
# for event in event_list:
#     record_list= event['content']
#     keys= None
#     for record in record_list:
#         keys= record['raw_data'].keys()
#         for item in keys:
#             record['raw_data'][item]= [record['raw_data'][item]]
#         record['raw_data']['date']=[record['date']]
#         record['raw_data']['DRZ'][0]= float(record['raw_data']['DRZ'][0])
#         lf_signal_point= pd.DataFrame(record['raw_data'])
#         if lf_signals is None:
#             lf_signals= lf_signal_point
#         else:
#             lf_signals= lf_signals.append(lf_signal_point,ignore_index= True)
# lf_signals['t_motor']= lf_signals['t_motor'].apply(np.float32)
# lf_signals.reset_index(inplace= True)
# lf_signals.to_csv(str(source_dir + 'lf2.csv'))


In [ ]:
#print (lf_signals.shape)

In [ ]:
## Extract Data source from all signals and create Training dataset
working_dir=  source_dir
os.chdir(working_dir)
results: np.ndarray= None
targets: np.ndarray= None
pipeline= None
reset= True
training_data= None
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df= pd.read_csv(file)
    signals= df
    signals= signals[selected_Columns]
    if training_data is None:
        training_data= signals
    else:
        training_data= pd.concat([training_data, signals])
training_data= training_data.to_numpy()
if pipeline  is None:
    try:
        if not reset:
            pipeline= load(str(preprocessor_dir + preprocessor_name))
            print('preprocessor is loaded successfully')
        else:
            pipeline= proposed_pipline
            pipeline= pipeline.fit(training_data)
            dump(pipeline, str(preprocessor_dir + preprocessor_name))
            print('preprocessor is saved successfully')
            #print('explained_variance_ratio_= ', pipeline.steps[1][1].explained_variance_ratio_)
    except FileNotFoundError as err:
        pipeline= proposed_pipline
        pipeline= pipeline.fit(training_data)
        dump(pipeline, str(preprocessor_dir + preprocessor_name))
        initial_type = [('float_input', FloatTensorType([None,  training_data.shape[1]]))]
        onx = convert_sklearn(pipeline, initial_types=initial_type)
        with open(str(preprocessor_dir + "preprocessor.onnx"), "wb") as f:
            f.write(onx.SerializeToString())
        print('preprocessor is saved successfully')
columns= selected_Columns.copy()
columns.append('t_welle')    
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df= pd.read_csv(file)
    signals= df
    signals= signals[selected_Columns].to_numpy()
    signals= pipeline.transform(signals)
    output= (df['t_welle'])
    correlations= df[['DRZ','t_bett','t_schlitten','t_spindle', 't_motor','t_welle']].corr()
    fig, ax= plt.subplots(figsize=(15,15))
    sn.heatmap(correlations, annot=True, vmin=-1, vmax=1)
    plt.show()
    ######Visualize Data######
    #create_histogram(df= df, name= file, features= columns)
    #visualize_data(df= df, name= file, features= columns)
    # temp_signal= pd.DataFrame(signals,columns=['p1'])
    # temp_signal['t_welle']= output
    #create_histogram(df= temp_signal, name= file, features= temp_signal.columns)
    #visualize_data(df= temp_signal, name= file, features= temp_signal.columns)
    ##########################
    partitions, target= generateDataSource(signal_input= signals, input_columns= selected_Columns, output_length= 1,signal_output= output, window= window, shift= shift)
    if results is None:
        results= partitions
        targets= target
    else:
        results= np.concatenate((results, partitions), axis= 0)
        targets= np.concatenate((targets, target), axis= 0)
    print('input: ', results.shape, ' output:', targets.shape)
#results= np.reshape(results, (-1, window,))
targets= np.reshape(targets, (-1,1))
#print('input: ', results.shape, ' output:', targets.shape)

In [ ]:
# ## Extract Data source from all signals
# working_dir=  testdata_dir
# os.chdir(working_dir)
# test_results: np.ndarray= None
# test_targets: np.ndarray= None
# for file in glob.glob('*.csv'):
#     if file == 'combined.csv':
#         continue
#     print('Current File: ', file)
#     df= pd.read_csv(file)
#     signals= df
#     signals= signals[selected_Columns]
#     signals= signals.to_numpy()
#     signals= pipeline.transform(signals)
#     #output= 1000000*(df['z_tcp_ok']).round(3)
#     output= (df['z_tcp_ok']).round(5)
#     partitions, target= generateDataSource(signal_input= signals, input_columns= selected_Columns, output_length= 1,signal_output= output, window= window, shift= shift)
#     if test_results is None:
#         test_results= partitions
#         test_targets= target
#     else:
#         test_results= np.concatenate((test_results, partitions), axis= 0)
#         test_targets= np.concatenate((test_targets, target), axis= 0)
#     print('input: ', test_results.shape, ' output:', test_targets.shape)

In [ ]:
batch_size= 50
checkpoint= ModelCheckpoint(filepath= model_dir, monitor= 'val_loss', verbose=1,
                                     save_best_only=True, save_weights_only= True, mode='min',save_freq='epoch',)
#tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
earlyStopping= EarlyStopping(monitor= 'val_loss', mode= 'min', patience=200,verbose=1)
model= build_Model(X_train= results,y_train= targets)
#model= build_Model_MLP(X_train= results,y_train= targets)
try:
    if not reset:
        model.load_weights(filepath= model_dir)
        print('Previous weights loaded Successfully')
except:
    print('No Previous weights')
print('input shape ', model.input_shape)
print('Output shape ',model.output_shape)
print(model.summary())
summary= model.fit(x= results, y= targets,shuffle= False, batch_size= batch_size,  epochs= 1000, validation_split= 0.3,#validation_data=(test_results, test_targets),
callbacks= [checkpoint, earlyStopping], verbose= 2, workers=32, use_multiprocessing= True)
model.save(str(model_dir+'model.h5'), save_format='h5')
input_signature= [tf.TensorSpec([None,results.shape[1], results.shape[2]],tf.float32, name= 'x')]
onnx_model, _= tf2onnx.convert.from_keras(model= model, input_signature= input_signature, opset= 10)
onnx.save_model(onnx_model, str(model_dir+'model.onnx'))
print (".onnx model saved successfully")

In [ ]:
#####Save Model as onnx from weights
from keras.models import load_model
import tf2onnx
import onnx
import tensorflow as tf
model= Sequential()
model= load_model(filepath= str(model_dir+'model.h5'), compile= False)
model.load_weights(model_dir)
model.save(str(model_dir+'best_model.h5'), save_format='h5')
input_signature= [tf.TensorSpec([None,results.shape[1], results.shape[2]],tf.float32, name= 'x')]
onnx_model, _= tf2onnx.convert.from_keras(model= model, input_signature= input_signature, opset= 10)
onnx.save_model(onnx_model, str(model_dir+'Best_model.onnx'))
print (".onnx model saved successfully")

In [ ]:
loss= summary.history['loss']
val_loss= summary.history['val_loss']
fig, ax= plt.subplots(figsize=(15,15))
plt.title('Training Results')
plt.grid()
plt.xlabel('Epochs')
plt.ylabel('MAE (mm)')
plt.plot(loss, label= 'Training Loss')
plt.plot(val_loss, label= 'validation Loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from keras.models import load_model
#from keras.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
#model=  Sequential()
#model= model.load_weights(model_dir,)#load_model(filepath= str(model_dir+'model.h5'), compile= False)
#print(model.summary())
# pred_train= model.predict(x_train)
# print ('mse= ',np.mean(mean_squared_error(y_train,pred_train)), ' mae= ', np.mean(mean_absolute_error(y_train,pred_train)), ' mape= ', np.mean(mean_absolute_percentage_error(y_train,pred_train)))
# pred_test= model.predict(x_test)
# print ('mse= ',np.mean(mean_squared_error(y_test,pred_test)), ' mae= ', np.mean(mean_absolute_error(y_test,pred_test)), ' mape= ', np.mean(mean_absolute_percentage_error(y_test,pred_test)))
pred= model.predict(results)
print (' For All Data mse= ',mean_squared_error(targets,pred), ' mae= ', mean_absolute_error(targets,pred))
fig, ax= plt.subplots(figsize=(15,15))
plt.title('Results of LSTM Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.grid()
# plt.plot(x_steps, upperline, label= 'Upper Bound', color='green')
plt.scatter(targets,pred, label= 'predictions_train', color= 'blue')
# plt.plot(x_steps, lowerline, label= 'lower Bound', color='green')
# plt.plot(x_steps, x_steps, label= 'Optimal line')
# plt.plot(x_steps, lowerline_actual, label= 'Actual lower Bound', color='red')
# plt.plot(x_steps, upperline_actual, label= 'Actual Upper Bound', color='red')
plt.legend()
plt.show()

In [ ]:
df= pd.DataFrame({'targets': np.reshape(targets,-1), 'pred': np.reshape(pred,-1)})
fig= ex.line(df,y=['targets', 'pred'])
#fig.update_layout(height=1200, width=1200, title_text="Laser-Data")
fig.show()

In [ ]:
diff= abs(df['targets'] - df['pred'])
fig= ex.line(y= diff)
#fig.update_layout(height=1200, width=1200, title_text="Laser-Data")
fig.show()
print ('Max= ', diff.max(),' Min= ', diff.min(), 'mean= ', diff.mean() )